In [ ]:
!pip install "ray[serve]"

In [1]:
import ray

In [ ]:
dataset = ray.data.read_parquet("nyc_taxi_2021.parquet")
dataset = dataset.drop_columns(['__index_level_0__'])
train_ds, test_ds = dataset.train_test_split(test_size=0.3)
print(train_ds)
print(test_ds)

(ServeReplica:default:OnlinePredictor pid=58582) ERROR 2025-03-06 10:56:32,938 default_OnlinePredictor dqh1myds 3bbf6b5f-614d-473b-b522-78ee43fffa98 -- Request failed.
(ServeReplica:default:OnlinePredictor pid=58582) Traceback (most recent call last):
(ServeReplica:default:OnlinePredictor pid=58582)   File "/Users/justin/miniconda3/envs/ai-computing/lib/python3.11/site-packages/ray/serve/_private/replica.py", line 472, in _handle_errors_and_metrics
(ServeReplica:default:OnlinePredictor pid=58582)     yield _status_code_callback
(ServeReplica:default:OnlinePredictor pid=58582)   File "/Users/justin/miniconda3/envs/ai-computing/lib/python3.11/site-packages/ray/serve/_private/replica.py", line 880, in _wrap_user_method_call
(ServeReplica:default:OnlinePredictor pid=58582)     yield status_code_callback
(ServeReplica:default:OnlinePredictor pid=58582)   File "/Users/justin/miniconda3/envs/ai-computing/lib/python3.11/site-packages/ray/serve/_private/replica.py", line 646, in handle_request_

In [3]:
from ray.train.xgboost import XGBoostTrainer
from ray.train import ScalingConfig, RunConfig

trainer = XGBoostTrainer(
    label_column="is_big_tip",
    scaling_config=ScalingConfig(num_workers=4, use_gpu=False),
    params={"objective": "binary:logistic"},
    datasets={"train": train_ds, "valid": test_ds},
    run_config=RunConfig(storage_path="/tmp/cluster_storage"),
)


In [4]:
from ray import tune
from ray.tune import Tuner, TuneConfig

tuner = Tuner(
    trainer,
    param_space={"params": {"max_depth": tune.randint(2, 12)}},
    tune_config=TuneConfig(num_samples=3, metric="valid-logloss", mode="min"),
    run_config=RunConfig(storage_path="/tmp/cluster_storgage/")
)
checkpoint = tuner.fit().get_best_result().checkpoint

(pid=58556) Running 0: 0.00 row [00:00, ? row/s]

(pid=58556) - split(4, equal=True) 1: 0.00 row [00:00, ? row/s]

(pid=58555) Running 0: 0.00 row [00:00, ? row/s]

(pid=58555) - split(4, equal=True) 1: 0.00 row [00:00, ? row/s]

(pid=58554) Running 0: 0.00 row [00:00, ? row/s]

...

(pid=58568) Running 0: 0.00 row [00:00, ? row/s]

(pid=58568) - split(4, equal=True) 1: 0.00 row [00:00, ? row/s]

(pid=58569) Running 0: 0.00 row [00:00, ? row/s]

(pid=58569) - split(4, equal=True) 1: 0.00 row [00:00, ? row/s]

2025-03-06 10:43:45,203	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/tmp/cluster_storgage/XGBoostTrainer_2025-03-06_10-43-16' in 0.0097s.
2025-03-06 10:43:45,209	INFO tune.py:1041 -- Total run time: 28.70 seconds (28.67 seconds for the tuning loop).


In [5]:
import xgboost
import pandas as pd

class OfflinePredictor:
    def __init__(self):
        self._model = xgboost.Booster()
        self._model.load_model(checkpoint.path + "/model.ubj")

    def __call__(self, batch: dict) -> dict:
        dmatrix = xgboost.DMatrix(pd.DataFrame(batch))
        prediction = self._model.predict(dmatrix)
        return {"prediction": prediction}

In [6]:
test_ds_features = test_ds.drop_columns(['is_big_tip'])
predicted_prob = test_ds_features.map_batches(OfflinePredictor, concurrency=2)
predicted_prob.take_batch(20)

2025-03-06 10:43:48,997	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-06_10-42-52_512282_58519/logs/ray-data
2025-03-06 10:43:48,998	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> ActorPoolMapOperator[MapBatches(drop_columns)->MapBatches(OfflinePredictor)] -> LimitOperator[limit=20]


Running 0: 0.00 row [00:00, ? row/s]

- MapBatches(drop_columns)->MapBatches(OfflinePredictor) 1: 0.00 row [00:00, ? row/s]

- limit=20 2: 0.00 row [00:00, ? row/s]

{'prediction': array([0.62117326, 0.6351201 , 0.53432137, 0.63928   , 0.55415016,
        0.5599472 , 0.6341473 , 0.628925  , 0.6034618 , 0.56432724,
        0.53527355, 0.5456829 , 0.6368325 , 0.61181074, 0.55794364,
        0.61181074, 0.54065686, 0.52323276, 0.49547666, 0.59156513],
       dtype=float32)}

## Serve the model

In [16]:
from ray import serve
from starlette.requests import Request
import pandas as pd
import json


@serve.deployment
class OnlinePredictor:
    def __init__(self, checkpoint):
        self._model = xgboost.Booster()
        self._model.load_model(checkpoint.path + "/model.ubj")

    async def __call__(self, request: Request) -> dict:
        req = await request.json()
        data = json.loads(req)
        dmatrix = xgboost.DMatrix(pd.DataFrame(data))
        prediction = self._model.predict(dmatrix)
        return {"prediction": prediction}

handle = serve.run(OnlinePredictor.bind(checkpoint=checkpoint))

INFO 2025-03-06 11:03:52,422 serve 58519 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.
INFO 2025-03-06 11:03:57,466 serve 58519 -- Application 'default' is ready at http://127.0.0.1:8000/.
INFO 2025-03-06 11:03:57,467 serve 58519 -- Deployed app 'default' successfully.


### Make request

In [19]:
import requests

sample_batch = test_ds_features.take_batch(10)
data = pd.DataFrame(sample_batch).to_json(orient="records")

requests.post("http://localhost:8000/", json=data).json()

2025-03-06 11:04:17,264	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-06_10-42-52_512282_58519/logs/ray-data
2025-03-06 11:04:17,265	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(drop_columns)] -> LimitOperator[limit=10]


Running 0: 0.00 row [00:00, ? row/s]

- MapBatches(drop_columns) 1: 0.00 row [00:00, ? row/s]

- limit=10 2: 0.00 row [00:00, ? row/s]

{'prediction': [0.6211732625961304,
  0.6351200938224792,
  0.5343213677406311,
  0.6392800211906433,
  0.5541501641273499,
  0.5599471926689148,
  0.6341472864151001,
  0.6289250254631042,
  0.6034618020057678,
  0.5643272399902344]}

### Shutdown

In [21]:
serve.shutdown()